In [ ]:
# File to investigate feature engineering

In [ ]:
import pyreadr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression
from sklearn.preprocessing import StandardScaler
from lifelines import CoxPHFitter
import featuretools as ft
from featuretools.primitives import (
    AddNumeric, SubtractNumeric, MultiplyNumeric, DivideNumeric,
    Absolute, Negate, Percentile
)

In [ ]:
df_complete = pyreadr.read_r("/home/jupyter-niclas/Approach_3_Incuding_HbA1c_trend/Feature_engineering/input_cox_trend.rds")
# Coverting R-file as panda
df = df_complete[None]
df.head()

In [ ]:
# === STEP 1: INITIAL CLEAN SETUP ===
# Load your DataFrame as df (with 'time_to_event' and 'event')
df = df.reset_index(drop=True)
df['id'] = df.index

# Separate survival targets
df_base = df.drop(columns=['time_to_event', 'event'])

In [ ]:
# STEP 2: CREATE ENTITYSET AND APPLY FEATURETOOLS
es = ft.EntitySet(id="survival_data")
es = es.add_dataframe(dataframe_name="df", dataframe=df_base, index="id")

feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name="df",
    max_depth=2,
    verbose=True,
    trans_primitives=[
        AddNumeric(), SubtractNumeric(), MultiplyNumeric(), DivideNumeric(),
        Absolute(), Negate(), Percentile()
    ],
    n_jobs=1
)

# Add survival targets back in
feature_matrix['time_to_event'] = df['time_to_event'].values
feature_matrix['event'] = df['event'].values

In [ ]:
# STEP 3: CLEAN FEATURE MATRIX
X = feature_matrix.drop(columns=['time_to_event', 'event'])
X = X.select_dtypes(include=[np.number])
X = X.replace([np.inf, -np.inf], np.nan)
X = X.dropna(axis=1, thresh=int(0.98 * len(X)))
X = X.fillna(X.mean())

# Prepare target vectors
y_duration = feature_matrix['time_to_event'].values.astype('float32')
y_event = feature_matrix['event'].values.astype('int32')

In [ ]:
# STEP 4: SPLIT THE DATA
X_train, X_temp, y_dur_train, y_dur_temp, y_evt_train, y_evt_temp = train_test_split(
    X, y_duration, y_event, test_size=0.4, random_state=42, stratify=y_event)
X_val, X_test, y_dur_val, y_dur_test, y_evt_val, y_evt_test = train_test_split(
    X_temp, y_dur_temp, y_evt_temp, test_size=0.5, random_state=1234, stratify=y_evt_temp)

In [ ]:
# STEP 5: FEATURE SELECTION & SCALING
vt = VarianceThreshold(threshold=0.01)
X_train_vt = vt.fit_transform(X_train)
X_val_vt = vt.transform(X_val)
X_test_vt = vt.transform(X_test)

vt_feature_names = X.columns[vt.get_support()]
selector = SelectKBest(score_func=f_regression, k=50)
X_train_selected = selector.fit_transform(X_train_vt, y_dur_train)
X_val_selected = selector.transform(X_val_vt)
X_test_selected = selector.transform(X_test_vt)

selected_feature_names = vt_feature_names[selector.get_support()]

# Scale the selected features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected).astype('float32')
X_val_scaled = scaler.transform(X_val_selected).astype('float32')
X_test_scaled = scaler.transform(X_test_selected).astype('float32')

In [ ]:
# === STEP 6: LIFELINES COXPH BASELINE (WITHOUT FEATURE ENGINEERING) ===
X_train_base = pd.DataFrame(X_train_scaled, columns=selected_feature_names)
train_base = X_train_base.copy()
train_base['duration'] = y_dur_train
train_base['event'] = y_evt_train

# Drop collinear features
corr_matrix = X_train_base.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.98)]
X_train_base_filtered = X_train_base.drop(columns=to_drop)

# Rebuild and fit baseline model
train_base_filtered = X_train_base_filtered.copy()
train_base_filtered['duration'] = y_dur_train
train_base_filtered['event'] = y_evt_train

cph_base = CoxPHFitter(penalizer=0.1)
cph_base.fit(train_base_filtered, duration_col='duration', event_col='event')
cph_base.print_summary()

In [ ]:
# === STEP 7: FINAL EVALUATION OF ENGINEERED FEATURES ===
print("\nTop features from engineered matrix used in final model:")
summary = cph_base.summary.abs().sort_values('coef', ascending=False)
print(summary[['coef', 'p']])

# Plot
plt.figure(figsize=(10, 6))
cph_base.plot(hazard_ratios=True)
plt.title("Feature Effects on Hazard (HR > 1 = increased risk)")
plt.tight_layout()
plt.show()

In [ ]:
# Alternative plot
# Sort features by coefficient
sorted_features = cph_base.params_.sort_values(ascending=True).index  
# Plot only those features in sorted order
plt.figure(figsize=(10, 6))
cph_base.plot(hazard_ratios=True, columns=sorted_features)
plt.tight_layout()
plt.savefig("engineered_feature_effects_sorted_trend.pdf", format="pdf")
plt.show()

In [ ]:
from pycox.models import CoxPH as PycoxCoxPH
from pycox.evaluation import EvalSurv
import torchtuples as tt

In [ ]:
# === STEP 8: TRAIN PYCOX MODEL WITH ENGINEERED FEATURES ===
X_val_tt = tt.tuplefy(X_val_scaled, (y_dur_val, y_evt_val))
X_test_tt = tt.tuplefy(X_test_scaled, (y_dur_test, y_evt_test))

net = tt.practical.MLPVanilla(in_features=X_train_scaled.shape[1], num_nodes=[32],
                              out_features=1, batch_norm=True, dropout=0.1)
model = PycoxCoxPH(net, tt.optim.Adam)
model.optimizer.set_lr(0.001)
model.fit(X_train_scaled, (y_dur_train, y_evt_train), batch_size=256, epochs=200,
          val_data=X_val_tt, verbose=True)
model.compute_baseline_hazards()

surv = model.predict_surv_df(X_test_scaled)
ev = EvalSurv(surv, y_dur_test, y_evt_test, censor_surv='km')
print(f"\nC-index on test set after feature engineering (PyCox): {ev.concordance_td():.3f}")


In [ ]:
# === STEP 9: COMPARE BASELINE AND ENGINEERED APPROACHES ===
from lifelines.utils import concordance_index

# Calculate baseline c-index
X_test_base = pd.DataFrame(X_test_scaled, columns=selected_feature_names).drop(columns=to_drop)
test_base = X_test_base.copy()
test_base['duration'] = y_dur_test
test_base['event'] = y_evt_test
baseline_c_index = concordance_index(
    test_base['duration'], -cph_base.predict_partial_hazard(test_base), test_base['event']
)

# Get PyCox c-index from earlier
engineered_c_index = ev.concordance_td()

# Plot comparison
plt.figure(figsize=(6, 4))
bars = plt.bar(['Baseline (lifelines)', 'Engineered (PyCox)'],
               [baseline_c_index, engineered_c_index], color=['skyblue', 'salmon'])
plt.ylabel("C-index")
plt.title("Model Comparison: Baseline vs Feature Engineered")
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, yval + 0.01, f"{yval:.3f}",
             ha='center', va='bottom')
plt.ylim(0, 1)
plt.tight_layout()
plt.savefig("Model_comparison:baseline_vs_feature_engineered_trend.jpg", dpi=300)
print("Plot saved as: Model_comparison:baseline_vs_feature_engineered_trend.jpg")
plt.show()

In [ ]:
# === STEP 10: FUNCTION TO EVALUATE C-INDEX BY FEATURE COUNT ===
from tqdm import tqdm

def evaluate_feature_counts(X_train_selected, X_val_selected, X_test_selected,
                            y_dur_train, y_evt_train, y_dur_val, y_evt_val,
                            y_dur_test, y_evt_test,
                            selected_feature_names, counts=[5, 10, 15, 20, 25, 30, 35, 40, 45, 50]):
    results = []
    print("Evaluating models with increasing feature counts:\n")
    for k in tqdm(counts, desc="Overall progress"):
        print(f"\n--- Running model with top {k} features ---")

        # Select top-k features
        feat_k = selected_feature_names[:k]
        idxs = [list(selected_feature_names).index(f) for f in feat_k]

        X_train_k = X_train_selected[:, idxs].astype('float32')
        X_val_k = X_val_selected[:, idxs].astype('float32')
        X_test_k = X_test_selected[:, idxs].astype('float32')

        # Scale
        scaler = StandardScaler()
        X_train_k = scaler.fit_transform(X_train_k).astype('float32')
        X_val_k = scaler.transform(X_val_k).astype('float32')
        X_test_k = scaler.transform(X_test_k).astype('float32')

        # Build and train PyCox model
        net = tt.practical.MLPVanilla(in_features=X_train_k.shape[1], num_nodes=[32],
                                      out_features=1, batch_norm=True, dropout=0.1)
        model = PycoxCoxPH(net, tt.optim.Adam)
        model.optimizer.set_lr(0.001)
        val_data = tt.tuplefy(X_val_k, (y_dur_val, y_evt_val))
        model.fit(X_train_k, (y_dur_train, y_evt_train), batch_size=256, epochs=100,
                  val_data=val_data, verbose=False)
        model.compute_baseline_hazards()

        # Evaluate
        surv = model.predict_surv_df(X_test_k)
        ev = EvalSurv(surv, y_dur_test, y_evt_test, censor_surv='km')
        c_index = ev.concordance_td()
        results.append((k, c_index))

    return pd.DataFrame(results, columns=['n_features', 'c_index'])

In [ ]:
# === STEP 11: RUN ANALYSIS AND PLOT ===
results_df = evaluate_feature_counts(
    X_train_selected, X_val_selected, X_test_selected,
    y_dur_train, y_evt_train, y_dur_val, y_evt_val,
    y_dur_test, y_evt_test,
    selected_feature_names
)

# Plot c-index vs. number of features
plt.figure(figsize=(8, 5))
plt.plot(results_df['n_features'], results_df['c_index'], marker='o', linestyle='-')
plt.xlabel("Number of Features Used")
plt.ylabel("C-index on Test Set")
plt.title("Impact of Number of Features on Model Performance")
plt.grid(True)
plt.tight_layout()
plt.savefig("Cox_impact_number_of_features_on_performance_trend.jpg", dpi=300)
print("Plot saved as: Cox_impact_number_of_features_on_performance_trend.jpg")
plt.show()

print(results_df)